# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import gmaps
import numpy as np
import pandas as pd
import time
import requests
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,San Patricio,75,US,1603654394,69,28.02,-97.52,84.99,14.99
1,Butembo,100,CD,1603654394,100,0.15,29.28,59.74,2.51
2,Qaanaaq,100,GL,1603654394,80,77.48,-69.36,12.72,12.08
3,Provideniya,100,RU,1603654394,91,64.38,-173.30,33.21,3.65
4,Albany,0,US,1603654159,43,42.60,-73.97,48.99,5.82
...,...,...,...,...,...,...,...,...,...
565,Portland,20,US,1603654338,28,45.52,-122.68,46.00,17.22
566,Manggar,100,ID,1603654475,81,-2.88,108.27,80.94,6.31
567,Christchurch,100,NZ,1603654475,93,-43.53,172.63,54.00,12.75
568,Magaria,64,NE,1603654475,20,13.00,8.91,83.07,6.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 #Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
93,Binga,0,ZW,1603654406,38,-17.62,27.34,78.30,1.88
113,Ridgecrest,0,US,1603654201,29,35.62,-117.67,73.00,10.00
131,Saint-Philippe,0,RE,1603654411,73,-21.36,55.77,70.00,6.93
156,Ocampo,0,MX,1603654414,19,21.65,-101.50,75.02,8.37
167,Kiryat Gat,0,IL,1603654416,83,31.61,34.76,77.00,5.82
229,Ahuatempan,0,MX,1603654426,43,18.42,-98.02,78.01,3.00
273,Sur,0,OM,1603654431,55,22.57,59.53,72.90,5.21
310,Agadez,0,NE,1603654437,20,19.75,10.25,73.90,8.23
390,Ambilobe,0,MG,1603654449,65,-13.20,49.05,73.08,7.20
426,Saint-Joseph,0,RE,1603654455,73,-21.37,55.62,70.00,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

NameError: name 'narrowed_city_df' is not defined

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [ ]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

   
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # get results
    results = response['results']
    
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']


    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    time.sleep(1)


print("--------End of Search---------")

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig
